# Table of Contents
[1. Install Playwright CLI](#1-install-playwright-cli)<br>
[2. Save session state](#2-save-session-state)<br>
[3. Configuring screenshots](#3-configuring-screenshots)<br>
[4. Playwright Extension methods](#4-playwright-extension-methods)<br>
[5. Running Playwright](#5-running-playwright)<br>
[6. Generate screenshot documentation](#6-generate-screenshot-documentation)<br>
[7. Example 1: Retrieve Solution rows and display the ](#7-example-1-retrieve-solution-rows-and-display-the-solutionid-of-default-solution)<br>
[8. Example 2 - Create new Opportunity record](#8-example-2---create-new-opportunity-record)<br>
[9. More about Playwright](#9-more-about-playwright)<br>

# 1. Install Playwright CLI

You can install [Playwright](https://playwright.dev/) as a dotnet global tool or as a npm global package. In this example we will install Playwright as a global tool using .NET CLI.

In [ ]:
#!powershell
dotnet tool install --global Microsoft.Playwright.CLI

# 2. Save session state

Any application that requires login, still saves the state (logged in or not) using cookies. If you start a new session with the previously captured session state, the application consideres you as having "logged in", even though you did not enter your credentials.

The code block below creates an empty project, installs the browsers (Chromium, Firefox and Webkit) and opens Chromium in "automation" mode. In this new browser window, navigate to the Power Apps application, login with you credentials, and close the window after you have logged in.

Run this code below only if you need to re-record your Power Apps login to create `auth.json`

In [ ]:
#!powershell

Get-Location;

# Create project
dotnet new console -n PlaywrightDemo
cd PlaywrightDemo

# Install dependencies, build project and download necessary browsers.
dotnet add package Microsoft.Playwright
dotnet build
playwright install

#start recording the browser
playwright codegen --save-storage=../../auth.json

cd..
Remove-Item ./PlaywrightDemo -Force -Recurse


After running the commands the session state will be captured in a file called `auth.json`. There are only two cookies needed for Power Apps to consider you as "logged in". They are

1. _CrmOwinAuth_
2. _ESTSAUTHPERSISTENT_

You can remove all the other information except these two. Below is a sample for `auth.json`. If you want to leave this file as it is, it is OK was well.

```json
{
  "cookies": [
    {
      "sameSite": "None",
      "name": "CrmOwinAuth",
      "value": "REMOVED",
      "domain": ".crm.dynamics.com",
      "path": "/",
      "expires": 1953359538.945023,
      "httpOnly": true,
      "secure": true
    },
    {
      "sameSite": "None",
      "name": "ESTSAUTHPERSISTENT",
      "value": "REMOVED",
      "path": "/",
      "expires": 1645602747.992615,
      "httpOnly": true,
      "secure": true
    }
  ]
}
```

Now we need to install the Playwright package on the Notebook, so that we can run some automation from this Notebook.

In [ ]:
#r "nuget:Microsoft.Playwright"

# 3. Configuring screenshots

[screenshots.config.json](screenshots.config.json) is where you specify the title of the screeshot, file name of the screenshot and URL to navigate to. Below is an example config.

```json
[
    {
        "title": "Home",
        "fileName": "Home",
        "url": "https://make.powerapps.com",
        "description": "This is the Maker portal."
    },
    {
        "title": "Customer Service App",
        "fileName": "Model Driven App",
        "url": "https://org.crm6.dynamics.com/main.aspx?appid=fc4884b2-6119-e811-a94f-000d3ae060f6",
        "description": "This is the customer service app used by all agents."
    }
]
```

The snippet below reads code from the config file and prompts the user for the URL of the model-driven app. This URL will be used in [Example 1](#7-example-1-retrieve-solution-rows-and-display-the-solutionid-of-default-solution) and [Example 2](#8-example-2---create-new-opportunity-record).

In [ ]:
using Microsoft.Playwright;
using System.Threading.Tasks;
using System.IO;
using System.Text.Json;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

record Config(string fileName, string url, string title, string description);
var config = JsonSerializer.Deserialize<Config[]>(File.ReadAllText("screenshots.config.json"));

var url = await GetInputAsync("Enter model driven app URL");
// var url = "https://env.crm6.dynamics.com/main.aspx?appid=f55f77d7-e05d-e911-a866-000d3ae0eb8e&forceUCI=1";

# 4. Playwright Extension methods

Now, let's setup some extension methods, to make it easy to use Playwright along with model-driven apps.

In [ ]:
using Microsoft.Playwright;
using static System.Text.Json.JsonElement;
using System;
using System.Threading.Tasks;
using System.Diagnostics;


public static async Task<ArrayEnumerator> RetrieveMultiple(this IPage page, RetrieveMultipleOptions options)
{
    var records = await page.EvaluateAsync(@"async(options) => {
        var records = await Xrm.WebApi.online.retrieveMultipleRecords(options.entity, options.query);
        return records; 
    }", options);
    var result = records.Value.GetProperty("entities").EnumerateArray();

    return result;
}

public static async Task<ObjectEnumerator> Retrieve(this IPage page, RetrieveOptions options)
{
    var records = await page.EvaluateAsync(@"async(options) => {
        var record = await Xrm.WebApi.online.retrieveRecord(options.entity, options.id, options.query);
        return record; 
    }", options);
    var result = records.GetValueOrDefault().EnumerateObject();

    return result;
}

public static async Task<Guid> Create(this IPage page, CreateOptions options)
{
    var records = await page.EvaluateAsync(@"async(options) => {
        var record = await Xrm.WebApi.online.createRecord(options.entity, options.data);
        return record; 
    }", options);
    var result = records.GetValueOrDefault().GetProperty("id").GetGuid();

    return result;
}

public static async Task<Guid> Update(this IPage page, UpdateOptions options)
{
    var records = await page.EvaluateAsync(@"async(options) => {
        var record = await Xrm.WebApi.online.updateRecord(options.entity, options.id, options.data);
        return record; 
    }", options);
    var result = records.GetValueOrDefault().GetProperty("id").GetGuid();

    return result;
}

public static async Task<Guid> Delete(this IPage page, DeleteOptions options)
{
    var records = await page.EvaluateAsync(@"async(options) => {
        var record = await Xrm.WebApi.online.deleteRecord(options.entity, options.id);
        return record; 
    }", options);
    var result = records.GetValueOrDefault().GetProperty("id").GetGuid();

    return result;
}

public static async Task<PowerAppsPage> OpenView(this IPage page, OpenOptions options)
{
    await page.WaitForFunctionAsync("window.Xrm !== undefined");

    await page.RunAndWaitForNavigationAsync(async () =>
    {
            await page.EvaluateAsync(@"async(options) => {
            await Xrm.Navigation.navigateTo({
                pageType: 'entitylist',
                entityName: options.entity
            })
        }", options);
    });

    await page.Locator($"div[data-id^=\"data-set-body-container\"]").WaitForAsync();
    return new PowerAppsPage(page, PageType.View);
}

public static async Task<PowerAppsFormPage> OpenRecord(this IPage page, OpenOptions options)
{
    await page.WaitForFunctionAsync("window.Xrm !== undefined");

    await page.RunAndWaitForNavigationAsync(async () =>
    {
        if (options.id != null)
        {
            await page.EvaluateAsync(@$"async(options) => {{
            await Xrm.Navigation.navigateTo({{
                pageType: 'entityrecord',
                entityName: options.entity,
                entityId: '{options.id.ToString().ToLower()}'
            }})
        }}", options);
        }
        else
        {
            await page.EvaluateAsync(@"async(options) => {
            await Xrm.Navigation.navigateTo({
                pageType: 'entityrecord',
                entityName: options.entity
            })
        }", options);
        }
    });

    await page.Locator($"div[id^=\"tab-section\"]").WaitForAsync();
    return new PowerAppsFormPage(page, PageType.Form);
}

public enum PageType
{
    View,
    Form
}

public class PowerAppsPage
{
    protected IPage _page;
    private PageType _pageType;
    public PowerAppsPage(IPage page, PageType pageType)
    {
        _page = page;
        _pageType = pageType;
    }
    public async Task<PowerAppsFormPage> NewRecord() {
        await _page.RunAndWaitForNavigationAsync(async () => await _page.ClickAsync("[aria-label=\"New\"]"));
        return new PowerAppsFormPage(_page, PageType.Form);
    }
}

public class PowerAppsFormPage : PowerAppsPage
{
    public PowerAppsFormPage(IPage page, PageType pageType) : base(page, pageType)
    {
    }

    public async void SetFieldValue(string field, string text) {
        await _page.ClickAsync($"input[id*=\"{field}\"]");
        await _page.FillAsync($"input[id*=\"{field}\"]", text);
    }    

    public async Task<IResponse> Save() => await _page.RunAndWaitForNavigationAsync(async () => await _page.ClickAsync("[aria-label=\"Save (CTRL+S)\"]"));

    public async Task<IResponse> SaveAndClose() => await _page.RunAndWaitForNavigationAsync(async () => await _page.ClickAsync("[aria-label=\"Save & Close\"]"));

    public async Task<Guid> GetRecordId()
    {
        await _page.WaitForFunctionAsync("window.Xrm !== undefined");
        var recordId = await _page.EvaluateAsync(@"()=>Xrm.Utility.getPageContext().input.entityId");
        var result = recordId.GetValueOrDefault().GetProperty("entityId").GetGuid();
        return result;
    }    
}

public record RetrieveOptions(string entity, Guid id, string query);
public record RetrieveMultipleOptions(string entity, string query);
public record CreateOptions(string entity, string data);
public record UpdateOptions(string entity, Guid id, string data);
public record DeleteOptions(string entity, Guid id);
public record OpenOptions(string entity, Guid? id = null);

# 5. Running Playwright

Since we now have the `auth.json` file with the persisted cookies we can start a new browser session with the pre-save cookie information. It will navigate us straight to the URLs mentioned in the config file, with no login screen at all. The cookies are valid for approximately 2 weeks. So, you might get error after the cookie expires. If that is the case, you need to run [Save Session State](#2-save-session-state) step again, to create a new `auth.json`. 

`auth.json` contains sensitive information, so it is ignored on `.gitignore`, so <ins>do not</ins> commit this to source control.

In [ ]:
using (var playwright = await Playwright.CreateAsync())
{
        //Use this if you are behind a proxy
        // var browser = await playwright.Firefox.LaunchAsync(new BrowserTypeLaunchOptions
        // {
        //     Headless = false,
        //     Proxy = new Proxy{Server=""}
        // });
        var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions{ Timeout=60000, Headless = false });        
        var context = await browser.NewContextAsync(new BrowserNewContextOptions { 
            StorageStatePath = "../auth.json",
            RecordVideoDir = "../videos",
            RecordVideoSize = new RecordVideoSize() { Width = 1920, Height = 1080 }
        });
        var page = await context.NewPageAsync();
        await page.SetViewportSizeAsync(1920, 1080);
        foreach (var c in config)
        {
            await page.GotoAsync(c.url);
            await page.WaitForLoadStateAsync(LoadState.NetworkIdle);
            await page.ScreenshotAsync(new PageScreenshotOptions { Path = $"../screenshots/{c.fileName}.png" });            
        }
        await context.CloseAsync();
}

When the above code has finished running you would also find a video of the recorded session inside the [videos](../videos/) folder.

# 6. Generate screenshot documentation

We can dynamically add the screenshot document and text using the Notebook itself by running the snippet below. The whole interaction is also recorded and a recording can be found in the [videos](./videos) folder.

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

var screenshot = div(
    h1(summary($"Screenshots as of {DateTime.Now.ToString()}")),
    details(
        config.Select(c => 
            div(
                h2(c.title),
                div(c.description),
                br,
                img[src:$"../screenshots/{c.fileName}.png",width:960, height: 540]
            )
        )
    )
);
display(screenshot);

//START: Remove this if you don't want the screenshot HTML to be saved into a file
if(!Directory.Exists("../documentation")) Directory.CreateDirectory("../documentation");

File.WriteAllText($"../documentation/{DateTime.Now.ToString("yyyy_MM_dd_HH_mm")}.html", @$"<!DOCTYPE html>
<html lang=""en"">
<head>
    <meta charset=""UTF-8"">
    <meta http-equiv=""X-UA-Compatible"" content=""IE=edge"">
    <meta name=""viewport"" content=""width=device-width, initial-scale=1.0"">
</head>
<body>
    {screenshot}
</body>
</html>");
//END

# 7. Example 1: Retrieve Solution rows and display the `solutionid` of _Default Solution_

You can use `EvaluateAsync` to execute arbitary JavaScript in the context of a page. The result from JavaScript can be utilised in C#.

In [ ]:
using (var playwright = await Playwright.CreateAsync())
{
    //Use this if you are behind a proxy
    // var browser = await playwright.Firefox.LaunchAsync(new BrowserTypeLaunchOptions
    // {
    //     Headless = false,
    //     Proxy = new Proxy{Server=""}
    // });
    var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions{Timeout=60000, Headless=false});        
    var context = await browser.NewContextAsync(new BrowserNewContextOptions { 
        StorageStatePath = "../auth.json"
    });
    var page = await context.NewPageAsync();
    await page.SetViewportSizeAsync(1920, 1080);
    await page.GotoAsync(url);
    await page.WaitForFunctionAsync("window.Xrm !== undefined");

    var appName = await page.EvaluateAsync("Xrm.Utility.getGlobalContext().getCurrentAppName()");

    var solutionName = "Default Solution";
    var solutions = await page.RetrieveMultiple(new RetrieveMultipleOptions("solution", $@"?$select=friendlyname&$filter=isvisible eq true and friendlyname eq '{solutionName}'"));    
    display(div(h3("Result"),div(ol(li($"The current app name: ", i(appName.Value.GetString())), li($"Solution Guid: ", i(solutions.First().GetProperty("solutionid").GetString()))))));
    await context.CloseAsync();
}

# 8. Example 2 - Create new Opportunity record

In [ ]:
using (var playwright = await Playwright.CreateAsync())
{
    //Use this if you are behind a proxy
    // var browser = await playwright.Firefox.LaunchAsync(new BrowserTypeLaunchOptions
    // {
    //     Headless = false,
    //     Proxy = new Proxy{Server=""}
    // });
    var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions{Timeout=60000, Headless=false}); 
    try
    {               
        var context = await browser.NewContextAsync(new BrowserNewContextOptions { 
            StorageStatePath = "../auth.json"     
        });
        var page = await context.NewPageAsync();
        await page.SetViewportSizeAsync(1920, 1080);
        await page.GotoAsync(url);

        var view = await page.OpenView(new OpenOptions("opportunity"));

        var record = await view.NewRecord();
        record.SetFieldValue("name", "Interactive Notebook");
        await record.Save();
        await page.WaitForFunctionAsync("Xrm.Utility.getPageContext().input.entityId != undefined");
        var recordId = await page.EvaluateAsync("Xrm.Utility.getPageContext().input.entityId");
        var appUrl = await page.EvaluateAsync("Xrm.Utility.getGlobalContext().getCurrentAppUrl()");
        display(div(span("New Opportunity record created -> "),a[href:$"{appUrl}&pagetype=entityrecord&etn=opportunity&id={recordId}"]("New Opportunity")));
    }
    catch (Exception ex)
    {
        Console.WriteLine(ex.Message);
    }
    finally
    {
        await browser.CloseAsync();
    }
}

# 9. More about Playwright

1. [Playwright documentation](https://playwright.dev/dotnet/docs/api/class-playwright)
2. [Playwright YouTube](https://www.youtube.com/channel/UC46Zj8pDH5tDosqm1gd7WTg)